# Informed search - the A* algorithm

Depth-first search and breadth-first search considered in the previous lesson are completely *blind* algorithms: they're only concerned whether the currently considered state is a goal state or not. They're unable to distinguish whether a state is easy or hard to reach, or whether it is near or far of the goal. This makes them very inefficient search algorithms. To allievate the issue, we introduce informed search algorithms. The information is given to an algorithm in two ways:

1. By using an *action cost* function $c(s,a)$, which, given a state $s$ and an action $a$ available in this state, returns its cost as a non-negative number.
2. By using a *heuristic* $h(s)$, which, given a state, estimates the lowest cost to reach a goal state from the given state.

Given a sequence of actions $a_1, \ldots, a_n$ and an initial state $s_1$, we can express the *total cost* of reaching the state $s_{n+1}$ by executing the sequence as:
$$ c(s_1, a_1, \ldots, a_{n-1}) = \sum_{i=1}^n c(s_i, a_i) $$
and the *expected cost* of the solution as the sum of the total cost and the estimate cost of reaching the goal from the state $s_{n+1}$
$$ f(s_1, a_1, \ldots, a_n) = c(s_1, a_1, \ldots, a_n) + h(s_{n+1}) $$

The heuristic function is a bit tricky, because we want it to have two properties:
* *Admissibility*: It must never *overestimate* the true cost of reaching the goal. 
* *Consistency*: Let $s$ be a state such that $a$ is an available action in this state and $s'$ is the state reached by executing this action. The heuristic should fulfil triangle inequality, that is, the estimated cost to reach the goal from $s$ should be no greater than the cost of executing the action $a$ + the estimated cost of reaching the goal from the new state.
$$ h(s) \leq c(s, a) + h(s') $$

One can prove that admissibility follows from consistency, but consistency is important only if there are multiple paths to reach the same state (i.e., we are searching in a graph, not in a tree). Otherwise, admissability is sufficient.

Lets extend the class `Problem` from the previous lesson with two new functions `action_cost` and `heuristic`, which correspond to the functions $c(s,a)$ and $h(s)$ described above.

In [ ]:
class Problem:
    @property
    def initial_state(self):
        ...
        
    def available_actions(self, state):
        ...        
        
    def do_action(self, state, action):
        ...
        return new_state
    
    def is_goal(self, state) -> bool:
        ...
        
    def action_cost(self, state, action) -> float:
        ...
        
    def heuristic(self, state, path) -> float:
        ...

To make a concrete example, lets revisit the vacuum world. 

![](aima-images/fig2_2.png)

Below, we assume a very simple model:
* Any action costs 1. This corresponds to searching for the shortest plan.
* The heuristic estimation is the number of fields which are still dirty. 


Lets consider the properties of the heuristic:
* Is is admissible? The heuristic value is equal to the number of 'Suck' actions that are yet to be executed and ignores the spatial aspect (i.e., moving between the rooms), thus never overestimating.
* Is it consistent? As a consequence of a single action the heuristic value can decrease by at most 1 (if the action happens to be 'Suck' and the room is dirty). The cost of any action is 1, so rewriting the triangle inequality we arrive at:
$$ h(s) \leq c(s, a) + h(s') = \begin{cases} 1 + (h(s)-1) & a=\text{'Suck' and the room was dirty} \\ 1 + h(s) & \text{otherwise} \end{cases} $$
* Is it the best we could have? By no means! We could include the spatial aspect.

In [ ]:
import copy
class VacuumProblem(Problem):
    @property
    def initial_state(self):
        return (0, (True, True))
    
    def available_actions(self, state):
        return ["Left", "Suck", "Right"]
        
    def do_action(self, state, action):
        robot, dirty = copy.deepcopy(state)
        if action == "Left":
            return (max(robot-1, 0), dirty)
        elif action == "Suck":
            new_dirty = list(dirty)
            new_dirty[robot] = False
            return (robot, tuple(new_dirty))
        elif action == "Right":
            return (min(robot+1, len(dirty)-1), dirty)        
        raise Exception('Invalid action')

    def is_goal(self, state) -> bool:
        return not any(state[1])
    
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state, path):
        return sum(state[1]) + len(path)

## Task 1: Implement the A* algorithm

To implement the A* algorithm you must have a priority queue. Luckily, Python comes with one, so you don't need to implement it by yourself. Then, the algorithm is very simple: 
1. Start with a queue containing a single item - the initial state
2. Repeat until the queue is not empty:
  1. Pick an item with the lowest expected cost
  2. If this is the goal, return the sequence of actions necessary to reach this state
  3. Otherwise, for each available action, create a new entry in the queue corresponding to the state reached after executing the action.
  
Guard the algorithm against infinite loops: if you already visited a state, you don't need to visit it again (if your heuristic is consistent).

In the cell below implement the algorithm in a similar manner as the BFS and DFS in the previous lesson: the sole argument is an object of the class Problem and the function should return a list of actions to achieve a goal state from the initial state.
If it is impossible to reach the goal, return `None`.
Count the number of states visited during the search and print in out before returning from the function, it will be useful later on to compare different heuristics.

In [ ]:
import heapq
import copy
def astar(problem: Problem):
    if problem.is_goal(problem.initial_state):
        return(problem.initial_state)
    root = (problem.heuristic(problem.initial_state, []), problem.initial_state, [])
    frontier = [root]
    heapq.heapify(frontier)
    explored = []
    states = 0
    while frontier:
        node = heapq.heappop(frontier)
        states += 1
        if problem.is_goal(node[1]):
            print('Number of states visited: ', states)
            return node
        explored.append(node[1])
        for action in problem.available_actions(node[1]):
            path = copy.deepcopy(node[2])
            path.append(action)
            child = (problem.heuristic(problem.do_action(node[1], action), path), problem.do_action(node[1], action) , path)
            if child[1] not in explored and child not in frontier:
                heapq.heappush(frontier,child)
            elif child in frontier:
                ix = frontier.index(child)
                incumbent = frontier[ix]
                if problem.heuristic(child[1], child[2]) < problem.heuristic(incumbent[1], incumbent[2]):
                    del frontier[ix]
                    heapq.heappush(frontier,child)
    return None

Now lets test your code in the vacuum world!

In [ ]:
astar(VacuumProblem())

Number of states visited:  4


(3, (1, (False, False)), ['Suck', 'Right', 'Suck'])

## Task 2: Variants of the vacuum world

Now lets consider a different take on the vacuum world in which the heuristic is not admissible and increases as the number of dirty fields decreases.

In [ ]:
class VacuumProblem1(VacuumProblem):
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state, path):
        return len(state[1]) - sum(state[1]) + len(path)
    
astar(VacuumProblem1())

Number of states visited:  7


(5, (1, (False, False)), ['Suck', 'Right', 'Suck'])

And another in which heuristic grossly overestimates the cost.

In [ ]:
class VacuumProblem2(VacuumProblem):
    def action_cost(self, state, action):
        return 1
    
    def heuristic(self, state, path):
        return 10 * sum(state[1]) + len(path)
    
astar(VacuumProblem2())

Number of states visited:  4


(3, (1, (False, False)), ['Suck', 'Right', 'Suck'])

**Which of the three heuristic functions (`VacuumProblem`, `VacuumProblem1`, `VacuumProblem2`) is the best? Is it the expected answer given the properties of the heuristics? If not, explain why an unorthodox approach works better.**

I'd say that the best heuristic out of those 3 is the first one, even though for this particular initial state it gives the same value as the first one. One would assume that putting high pressure on looking for states where the vacuum can reach it's goal is the most intuitive approach. In my opinion simplicity is the key, and since for such a simple agent the algorithm will deliver good results for the first approach there is no need to change it.

## Task 3: 8-puzzle problem

Recall the 8-puzzle problem from the previous lesson. Reuse your code and implement an extended version assuming that each action costs 1. Propose 3 (at least) admissible heuristics. This time don't change the initial state, your solution should be capable enough to solve this.

![](aima-images/fig3_4.png)

In [ ]:
import copy
class PuzzleProblem(Problem):
    @property
    def initial_state(self):
        return [7, 2, 4, 5, 0, 6, 8, 3, 1]
        
    def available_actions(self, state):
        pos = state.index(0)
        if(pos == 0):
          return ['DOWN', 'RIGHT']
        if(pos == 1):
          return ['DOWN', 'LEFT', 'RIGHT']
        if(pos == 2):
          return ['DOWN', 'LEFT']
        if(pos == 3):
          return ['DOWN', 'UP', 'RIGHT']
        if(pos == 4):
          return ['DOWN', 'UP', 'LEFT', 'RIGHT']
        if(pos == 5):
          return ['DOWN', 'UP', 'LEFT']
        if(pos == 6):
          return ['UP', 'RIGHT']
        if(pos == 7):
          return ['UP', 'LEFT', 'RIGHT']
        if(pos == 8):
          return ['UP', 'LEFT']

    def do_action(self, state, action):
        new_state = copy.deepcopy(state)
        pos = state.index(0)
        if action == 'LEFT':
          new_pos = pos - 1
        if action == 'RIGHT':
          new_pos = pos + 1
        if action == 'UP':
          new_pos = pos - 3
        if action == 'DOWN':
          new_pos = pos + 3
        new_state[pos], new_state[new_pos] = new_state[new_pos], new_state[pos]
        return new_state

    def is_goal(self, state) -> bool:
      return state == [0, 1, 2, 3, 4, 5, 6, 7, 8]

    def action_cost(self, state, action) -> float:
        return 1
        
    def heuristic(self, state, path) -> float:
        incorrect = 0
        for i in range(9):
            if i // 3 != state.index(i) // 3:
                incorrect += 1
            if i % 3 != state.index(i) % 3:
                incorrect += 1
        return incorrect + len(path)

**Prove that this heuristic is admissible.**

The first heuristic counts the number of tiles which are not in their correct rows and/or collumns.
This is an admissible heuristic, since every tile that is out of column or out of row must be moved at least once and every tile that is both out of column and out of row must be moved at least twice.

In [ ]:
import math
class PuzzleProblem1(PuzzleProblem):
    def heuristic(self, state, path) -> float:  
        distance = 0
        for i in range(9):
            distance += math.sqrt((abs((i % 3) - (state.index(i) % 3)) ** 2) + (abs((i // 3) - (state.index(i) // 3)) **2))
        return distance + len(path)

**Prove that this heuristic is admissible.**

The second heuristic value is equal to the euclidean distance of the current placement of a value and it's correct placement. This is an admissible heuristic, since in every move, one tile can only move closer to its goal by one step and the eucledian distance is never greater than the number of steps required to move a tile to its goal position.

In [ ]:
class PuzzleProblem2(PuzzleProblem):
    def heuristic(self, state, path) -> float:
        distance = 0
        for i in range(9):
            distance += abs((i % 3) - (state.index(i) % 3)) + abs((i // 3) - (state.index(i) // 3))
        return distance + len(path)

**Prove that this heuristic is admissible.**

The third heuristic value is equal to the manhattan distance of the current placement of a value and it's correct placement. This is an admissible heuristic, since in every move, one tile can only move closer to its goal by one step.


Run your heuristics on the given problem.

In [ ]:
n = astar(PuzzleProblem())
n
print(len(n[2]))

Number of states visited:  11200
26


In [ ]:
plan0 = astar(PuzzleProblem())
print(len(plan0[2]))
plan1 = astar(PuzzleProblem1())
print(len(plan1[2]))
plan2 = astar(PuzzleProblem2())
print(len(plan2[2]))

print("Is plan0==plan1?", plan0 == plan1)
print("Is plan0==plan2?", plan0 == plan2)
print("Is plan1==plan2?", plan1 == plan2)

Number of states visited:  11200
26
Number of states visited:  8882
26
Number of states visited:  2930
26
Is plan0==plan1? True
Is plan0==plan2? True
Is plan1==plan2? True


**Which of the heuristics is the best for this task? Why is that?**

The best heuristic for this task is Manhattan distance. Even though all 3 of them give the same path, the third one does it the fastest (as can be seen by the number of states visited).That is because if a tile could move one square in any direction, even onto an occupied square, then the third heuristic would give the exact length of the shortest solution.

------------
The pictures and the description of 8-puzzle are from "Artificial Intelligence: A Modern Approach" 3rd ed.